In [3]:
import pandas as pd
import numpy as np

# List of CSV files
csv_files = ['POI_FINAL_Museum.csv', 'POI_FINAL_Restaurant.csv', 'POI_FINAL_Travel_Agency.csv']

weights = {
        'Distance': 0.2,
        'Popularity': 0.1,
        'Ratings': 0.3,
        'Total Ratings': 0.1,
        'Price($)': 0.3
}
    
for file in csv_files:
    # Read the CSV file
    data = pd.read_csv(file, encoding='latin1')
    
    # Normalize the data
    for criterion in weights.keys():
        if criterion == 'Price($)':
            data[criterion] = (data[criterion].max() - data[criterion]) / (data[criterion].max() - data[criterion].min())
        else:
            data[criterion] = (data[criterion] - data[criterion].min()) / (data[criterion].max() - data[criterion].min())
    
    # Calculate the weighted sum using Promethee
    weighted_sum = np.zeros(len(data))
    for i in range(len(data)):
        for j in range(len(data)):
            if i != j:
                pref_ij = np.sum([int(data.iloc[i][criterion] > data.iloc[j][criterion]) * weights[criterion] for criterion in weights.keys()])
                pref_ji = np.sum([int(data.iloc[j][criterion] > data.iloc[i][criterion]) * weights[criterion] for criterion in weights.keys()])
                weighted_sum[i] += pref_ij - pref_ji
    
    # Add the weighted sums to the DataFrame
    data['Weighted Sum'] = weighted_sum
    
    # Rank the alternatives based on the weighted sum
    data['Rank'] = data['Weighted Sum'].rank(ascending=False, method='min')
    
    # Display the result
    print(f"Results for {file}:")
    # Truncate the name
    data['Name'] = data['Name'].apply(lambda x: ' '.join(x.split()[:2]))
    print(data[['Name', 'Latitude', 'Longitude', 'Weighted Sum', 'Rank']])
    print("\n")


Results for POI_FINAL_Museum.csv:
                Name   Latitude  Longitude  Weighted Sum  Rank
0            Jane A.  40.767370 -73.981125 -4.000000e-01   6.0
1           Spyscape  40.765198 -73.983618  2.000000e-01   4.0
2         The Africa  40.796289 -73.949378 -5.000000e+00  10.0
3    Madame Tussauds  40.756539 -73.988521 -3.400000e+00   9.0
4           New York  40.752895 -73.977589 -6.000000e-01   7.0
5         Center for  40.752768 -73.992304 -2.600000e+00   8.0
6  Fashion Institute  40.747304 -73.994797  4.200000e+00   2.0
7       Hebrew Union  40.728807 -73.994549  1.665335e-16   5.0
8         Lower East  40.724864 -73.992735  5.400000e+00   1.0
9            The New  40.726021 -73.999809  2.200000e+00   3.0


Results for POI_FINAL_Restaurant.csv:
                    Name   Latitude  Longitude  Weighted Sum  Rank
0       Daily Provisions  40.782225 -73.978646           0.3   5.0
1          Levain Bakery  40.781511 -73.979099           1.2   3.0
2            Sushi Kaito  40.779